##  1. Download and Explore Dataset

In [1]:
!pip install bs4
#!pip install plotly

import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library



print('Libraries imported.')

Solving environment: done

# All requested packages already installed.

Libraries imported.


In [2]:
url="https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"

In [3]:
Toronto_data= requests.get(url).text 

In [4]:
from bs4 import BeautifulSoup
soup = BeautifulSoup(Toronto_data, 'html5lib')

In [5]:
import re
tables = soup.find_all('table')
len(tables)
for index,table in enumerate(tables):
    if ("M" in str(table)):   # character M is the one we see in the Table
        table_index = index
print(table_index)

2


In [6]:
table_contents=[]
table=soup.find('table')
for row in table.findAll('td'):
    cell = {}
    if row.span.text=='Not assigned':
        pass
    else:
        cell['PostalCode'] = row.p.text[:3]
        cell['Borough'] = (row.span.text).split('(')[0]
        cell['Neighborhood'] = (((((row.span.text).split('(')[1]).strip(')')).replace(' /',',')).replace(')',' ')).strip(' ')
        table_contents.append(cell)

# print(table_contents)

neighborhoods=pd.DataFrame(table_contents)
neighborhoods['Borough']= neighborhoods['Borough'].replace({'Downtown TorontoStn A PO Boxes25 The Esplanade':'Downtown Toronto Stn A',
                                             'East TorontoBusiness reply mail Processing Centre969 Eastern':'East Toronto Business',
                                             'EtobicokeNorthwest':'Etobicoke Northwest','East YorkEast Toronto':'East York/East Toronto',
                                             'MississaugaCanada Post Gateway Processing Centre':'Mississauga'})

In [7]:
neighborhoods.shape

(103, 3)

####Data Wrangling

In [8]:
 neighborhoods['PostalCode'].describe()

count     103
unique    103
top       M4G
freq        1
Name: PostalCode, dtype: object

In [21]:
 neighborhoods.head(15)

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.686588,-79.409996
1,M4A,North York,Victoria Village,43.731540,-79.314280
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.659743,-79.361561
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.722940,-79.431160
4,M7A,Queen's Park,Ontario Provincial Government,43.725090,-79.482090
5,M9A,Etobicoke,Islington Avenue,43.663280,-79.531537
6,M1B,Scarborough,"Malvern, Rouge",43.810230,-79.220380
7,M3B,North York,Don Mills North,43.705685,-79.333857
8,M4B,East York,"Parkview Hill, Woodbine Gardens",43.706260,-79.300910
9,M5B,Downtown Toronto,"Garden District, Ryerson",43.657940,-79.375620


In [10]:
print('The dataframe has {} boroughs and {} neighborhoods.'.format(
        len(neighborhoods['Borough'].unique()),
        neighborhoods.shape[0]))

The dataframe has 15 boroughs and 103 neighborhoods.


In [11]:
address = 'Toronto,ON'

geolocator = Nominatim(user_agent="marcohf@yahoo.com")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto City are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto City are 43.6534817, -79.3839347.


In [12]:
! pip install geocoder

In [13]:
import geocoder

# Defining a function to get coordinates

def get_latlng(neighborhood):
    # initialize your variable to None
    lat_lng_coords = None
    # loop until you get the coordinates
    while(lat_lng_coords is None):
        g = geocoder.arcgis('{}, Toronto,ON'.format(neighborhood))
        lat_lng_coords = g.latlng
    return lat_lng_coords


In [14]:
 neighborhoods.head()

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Queen's Park,Ontario Provincial Government


In [15]:
# Call the function to get the coordinates, store in a new list using list comprehension
coords = [ get_latlng(neighborhood) for neighborhood in  neighborhoods["Neighborhood"].tolist()]

In [16]:
coords[0:5]

[[43.6865884896713, -79.40999620161057],
 [43.73154000000005, -79.31427999999994],
 [43.659742848554075, -79.36156058002041],
 [43.72294000000005, -79.43115999999998],
 [43.72509000000008, -79.48208999999997]]

In [17]:
# Create temporary dataframe to populate the coordinates into Latitude and Longitude
df_coords = pd.DataFrame(coords, columns=['Latitude', 'Longitude'])
# Merge the coordinates into the original dataframe
neighborhoods['Latitude'] = df_coords['Latitude']
neighborhoods['Longitude'] = df_coords['Longitude']
print( neighborhoods.shape)
neighborhoods.head(5)

(103, 5)


,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.686588,-79.409996
1,M4A,North York,Victoria Village,43.731540,-79.314280
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.659743,-79.361561
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.722940,-79.431160
4,M7A,Queen's Park,Ontario Provincial Government,43.725090,-79.482090


In [86]:
print('The dataframe has {} boroughs and {} neighborhoods.'.format(
        len( neighborhoods['Borough'].unique()),
         neighborhoods.shape[0]
    )
)

The dataframe has 15 boroughs and 103 neighborhoods.


### Create  a map only for Toronto Down Town Borough

### Create a map of Toronto with neighborhoods superimposed on top.¶

In [87]:
# create map of Toronto using latitude and longitude values
map_Toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip( neighborhoods['Latitude'], neighborhoods['Longitude'],  neighborhoods['Borough'],  neighborhoods['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_Toronto)  
    
map_Toronto

In [18]:
# Create a slice from  Down Town Toronto 

In [25]:
DownTown_data = neighborhoods[neighborhoods['Borough'] == 'Downtown Toronto'].reset_index(drop=True)
DownTown_data.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.659743,-79.361561
1,M5B,Downtown Toronto,"Garden District, Ryerson",43.657940,-79.375620
2,M5C,Downtown Toronto,St. James Town,43.671100,-79.373590
3,M5E,Downtown Toronto,Berczy Park,43.648110,-79.375170
4,M5G,Downtown Toronto,Central Bay Street,43.665283,-79.387556


In [28]:
address = 'Downtown Toronto, ON'

geolocator = Nominatim(user_agent="Toronto_dtwon")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Manhattan are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Manhattan are 43.6563221, -79.3809161.


In [29]:
# create map of Toronto Down Town using latitude and longitude values
map_dtwon = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(DownTown_data['Latitude'], DownTown_data['Longitude'], DownTown_data['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_dtwon)  
    
map_dtwon

## Define Foursquare Credentials and Version

In [30]:
CLIENT_ID = 'JRWH0X0QSE5RUZSK2XMC5ZJWHOBIN51ITQJ2CD4NZMAGN0Z3' # your Foursquare ID
CLIENT_SECRET = 'LBQZKN1PEPN5MWGFSFKTUB24PP2I3BQJ3I3PU5YEHQREJBVP' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 100 # A default Foursquare API limit value

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: JRWH0X0QSE5RUZSK2XMC5ZJWHOBIN51ITQJ2CD4NZMAGN0Z3
CLIENT_SECRET:LBQZKN1PEPN5MWGFSFKTUB24PP2I3BQJ3I3PU5YEHQREJBVP


### Let's explore the first neighborhood in our dataframe.¶

In [31]:
DownTown_data.loc[0, 'Neighborhood']

'Regent Park, Harbourfront'

#### Get the neighborhood's latitude and longitude values.

In [32]:
neighborhood_latitude =  DownTown_data.loc[0, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = DownTown_data.loc[0, 'Longitude'] # neighborhood longitude value

neighborhood_name =  DownTown_data.loc[0, 'Neighborhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of Regent Park, Harbourfront are 43.659742848554075, -79.36156058002041.


#### Now, let's get the top 100 venues that are in Marble Hill within a radius of 500 meters.
First, let's create the GET request URL. Name your URL url.

In [33]:
LIMIT=100 
radius=500

url= 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)
url # display URL





'https://api.foursquare.com/v2/venues/explore?&client_id=JRWH0X0QSE5RUZSK2XMC5ZJWHOBIN51ITQJ2CD4NZMAGN0Z3&client_secret=LBQZKN1PEPN5MWGFSFKTUB24PP2I3BQJ3I3PU5YEHQREJBVP&v=20180605&ll=43.659742848554075,-79.36156058002041&radius=500&limit=100'

In [34]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '60f76f18d490824a4c1e571e'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': 'Open now', 'key': 'openNow'}]},
  'headerLocation': 'Regent Park',
  'headerFullLocation': 'Regent Park, Toronto',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 22,
  'suggestedBounds': {'ne': {'lat': 43.66424285305408,
    'lng': -79.35535201328253},
   'sw': {'lat': 43.65524284405407, 'lng': -79.36776914675829}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '5092fe87e4b0476c6e7375e2',
       'name': 'Regent Park Aquatic Centre',
       'location': {'address': '640 Dundas St. East',
        'crossStreet': 'at Sumach St.',
        'lat': 43.6606003130796,
        'lng': -79.36139162915349,
        'lab

In [36]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

### Now we are ready to clean the json and structure it into a pandas dataframe.

In [40]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/ipykernel_launcher.py:3: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  This is separate from the ipykernel package so we can avoid doing imports until


,name,categories,lat,lng
0,Regent Park Aquatic Centre,Pool,43.660600,-79.361392
1,Sumach Espresso,Coffee Shop,43.658135,-79.359515
2,Daniels Spectrum,Performing Arts Venue,43.660137,-79.361808
3,Paintbox Bistro,Restaurant,43.660050,-79.362855
4,Sukhothai,Thai Restaurant,43.658444,-79.365681


In [38]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

22 venues were returned by Foursquare.


## 2. Explore Neighborhoods in Toronto Down Town

#### Let's create a function to repeat the same process to all the neighborhoods in Manhattan¶

In [41]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [55]:

Downtown_venues= getNearbyVenues(names=DownTown_data['Neighborhood'],
                                   latitudes=DownTown_data['Latitude'],
                                   longitudes=DownTown_data['Longitude']
                                  )

Regent Park, Harbourfront
Garden District, Ryerson
St. James Town
Berczy Park
Central Bay Street
Christie
Richmond, Adelaide, King
Harbourfront East, Union Station, Toronto Islands
Toronto Dominion Centre, Design Exchange
Commerce Court, Victoria Hotel
University of Toronto, Harbord
Kensington Market, Chinatown, Grange Park
CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport
Rosedale
St. James Town, Cabbagetown
First Canadian Place, Underground city
Church and Wellesley


In [56]:
print(Downtown_venues.shape)
Downtown_venues.head()

(1158, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Regent Park, Harbourfront",43.659743,-79.361561,Regent Park Aquatic Centre,43.660600,-79.361392,Pool
1,"Regent Park, Harbourfront",43.659743,-79.361561,Sumach Espresso,43.658135,-79.359515,Coffee Shop
2,"Regent Park, Harbourfront",43.659743,-79.361561,Daniels Spectrum,43.660137,-79.361808,Performing Arts Venue
3,"Regent Park, Harbourfront",43.659743,-79.361561,Paintbox Bistro,43.660050,-79.362855,Restaurant
4,"Regent Park, Harbourfront",43.659743,-79.361561,Sukhothai,43.658444,-79.365681,Thai Restaurant


### Let's check how many venues were returned for each neighborhood

In [58]:
Downtown_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Berczy Park,100,100,100,100,100,100
"CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport",76,76,76,76,76,76
Central Bay Street,41,41,41,41,41,41
Christie,42,42,42,42,42,42
Church and Wellesley,80,80,80,80,80,80
"Commerce Court, Victoria Hotel",100,100,100,100,100,100
"First Canadian Place, Underground city",100,100,100,100,100,100
"Garden District, Ryerson",82,82,82,82,82,82
"Harbourfront East, Union Station, Toronto Islands",91,91,91,91,91,91


### Let's find out how many unique categories can be curated from all the returned venues

In [59]:
print('There are {} uniques categories.'.format(len(Downtown_venues['Venue Category'].unique())))

There are 206 uniques categories.


## 3. Analyze Each Neighborhood

In [64]:
# one hot encoding
Downtown_onehot = pd.get_dummies(Downtown_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
Downtown_onehot['Neighborhood'] = Downtown_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [Downtown_onehot.columns[-1]] + list(Downtown_onehot.columns[:-1])
Downtown_onehot = Downtown_onehot[fixed_columns]
Downtown_onehot.head()

,Neighborhood,Adult Boutique,American Restaurant,Animal Shelter,Antique Shop,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Dealership,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Basketball Stadium,Bed & Breakfast,Beer Bar,Beer Store,Belgian Restaurant,Bistro,Bookstore,Botanical Garden,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Stop,Butcher,Café,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Gym,College Rec Center,College Theater,Comedy Club,Comfort Food Restaurant,Comic Shop,Concert Hall,Convenience Store,Cosmetics Shop,Costume Shop,Creperie,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Electronics Store,Escape Room,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish & Chips Shop,Fish Market,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Furniture / Home Store,Gaming Cafe,Garden,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Hawaiian Restaurant,Health & Beauty Service,Historic Site,Hobby Shop,Hookah Bar,Hospital,Hostel,Hotel,IT Services,Ice Cream Shop,Indian Restaurant,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Lake,Latin American Restaurant,Library,Light Rail Station,Liquor Store,Lounge,Market,Martial Arts School,Mattress Store,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Movie Theater,Museum,Music Venue,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Pakistani Restaurant,Paper / Office Supplies Store,Park,Pastry Shop,Performing Arts Venue,Peruvian Restaurant,Pet Store,Pharmacy,Pie Shop,Pizza Place,Playground,Plaza,Poke Place,Pool,Portuguese Restaurant,Poutine Place,Pub,Ramen Restaurant,Record Shop,Rental Car Location,Restaurant,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Smoke Shop,Smoothie Shop,Snack Place,South American Restaurant,Souvlaki Shop,Spa,Spanish Restaurant,Speakeasy,Sporting Goods Shop,Sports Bar,Sri Lankan Restaurant,Steakhouse,Street Art,Strip Club,Supermarket,Sushi Restaurant,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tea Room,Thai Restaurant,Theater,Theme Restaurant,Thrift / Vintage Store,Toy / Game Store,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Wine Bar,Yoga Studio
0,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0

In [65]:
#And let's examine the new dataframe size.

Downtown_onehot.shape

(1158, 207)

### Next, let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [66]:
Downtown_grouped = Downtown_onehot.groupby('Neighborhood').mean().reset_index()
Downtown_grouped

,Neighborhood,Adult Boutique,American Restaurant,Animal Shelter,Antique Shop,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Dealership,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Basketball Stadium,Bed & Breakfast,Beer Bar,Beer Store,Belgian Restaurant,Bistro,Bookstore,Botanical Garden,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Stop,Butcher,Café,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Gym,College Rec Center,College Theater,Comedy Club,Comfort Food Restaurant,Comic Shop,Concert Hall,Convenience Store,Cosmetics Shop,Costume Shop,Creperie,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Electronics Store,Escape Room,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish & Chips Shop,Fish Market,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Furniture / Home Store,Gaming Cafe,Garden,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Hawaiian Restaurant,Health & Beauty Service,Historic Site,Hobby Shop,Hookah Bar,Hospital,Hostel,Hotel,IT Services,Ice Cream Shop,Indian Restaurant,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Lake,Latin American Restaurant,Library,Light Rail Station,Liquor Store,Lounge,Market,Martial Arts School,Mattress Store,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Movie Theater,Museum,Music Venue,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Pakistani Restaurant,Paper / Office Supplies Store,Park,Pastry Shop,Performing Arts Venue,Peruvian Restaurant,Pet Store,Pharmacy,Pie Shop,Pizza Place,Playground,Plaza,Poke Place,Pool,Portuguese Restaurant,Poutine Place,Pub,Ramen Restaurant,Record Shop,Rental Car Location,Restaurant,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Smoke Shop,Smoothie Shop,Snack Place,South American Restaurant,Souvlaki Shop,Spa,Spanish Restaurant,Speakeasy,Sporting Goods Shop,Sports Bar,Sri Lankan Restaurant,Steakhouse,Street Art,Strip Club,Supermarket,Sushi Restaurant,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tea Room,Thai Restaurant,Theater,Theme Restaurant,Thrift / Vintage Store,Toy / Game Store,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Wine Bar,Yoga Studio
0,Berczy Park,0.00000,0.010000,0.000000,0.01,0.020000,0.000000,0.000000,0.000000,0.000000,0.010000,0.01,0.050000,0.000000,0.000000,0.000000,0.000000,0.020000,0.000000,0.000000,0.010000,0.010000,0.000000,0.00,0.020000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.050000,0.000000,0.020000,0.000000,0.00,0.01,0.000000,0.020000,0.080000,0.000000,0.000000,0.000000,0.00000,0.000000,0.010000,0.000000,0.010000,0.000000,0.010000,0.00,0.020000,0.00,0.000000,0.010000,0.010000,0.000000,0.010000,0.000000,0.00000,0.0000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.020000,0.000000,0.000000,0.000000,0.010000,0.000000,0.000000,0.000000,0.01,0.010000,0.00,0.000000,0.000000,0.000000,0.0000,0.030000,0.0000,0.000000,0.00,0.000000,0.010000,0.000000,0.010000,0.030000,0.000000,0.000000,0.000000,0.00000,0.0000,0.00000,0.00000,0.000000,0.010000,0.030000,0.000000,0.000000,0.000000,0.00000,0.000000,0.060000,0.040000,0.010000,0.000000,0.00000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.010000,0.0

In [68]:
# let's check the new size of the DF
Downtown_grouped.shape

(17, 207)

### Let's print each neighborhood along with the top 5 most common venues

In [69]:
num_top_venues = 5

for hood in Downtown_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = Downtown_grouped[Downtown_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Berczy Park----
                venue  freq
0         Coffee Shop  0.08
1          Restaurant  0.06
2  Italian Restaurant  0.06
3                Café  0.05
4              Bakery  0.05


----CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport----
                venue  freq
0         Coffee Shop  0.08
1          Restaurant  0.07
2  Italian Restaurant  0.07
3            Beer Bar  0.04
4               Hotel  0.03


----Central Bay Street----
                      venue  freq
0          Sushi Restaurant  0.10
1  Mediterranean Restaurant  0.05
2       Japanese Restaurant  0.05
3            Adult Boutique  0.02
4   Health & Beauty Service  0.02


----Christie----
                venue  freq
0  Italian Restaurant  0.07
1   Indian Restaurant  0.05
2         Coffee Shop  0.05
3          Restaurant  0.05
4    Sushi Restaurant  0.05


----Church and Wellesley----
                 venue  freq
0          Coffee Shop  0.10
1  Japanese Restau

### Let's put that into a pandas dataframe
First, let's write a function to sort the venues in descending order.

In [70]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

#### Now let's create the new dataframe and display the top 10 venues for each neighborhood.

In [78]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
Downtown_venues_sorted = pd.DataFrame(columns=columns)
Downtown_venues_sorted['Neighborhood'] = Downtown_grouped['Neighborhood']

for ind in np.arange(Downtown_grouped.shape[0]):
    Downtown_venues_sorted.iloc[ind, 1:] = return_most_common_venues(Downtown_grouped.iloc[ind, :], num_top_venues)

Downtown_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Berczy Park,Coffee Shop,Italian Restaurant,Restaurant,Bakery,Café,Japanese Restaurant,Seafood Restaurant,Gastropub,Gym,Hotel
1,"CN Tower, King and Spadina, Railway Lands, Har...",Coffee Shop,Restaurant,Italian Restaurant,Beer Bar,Yoga Studio,Bar,Park,Café,Indian Restaurant,Hotel
2,Central Bay Street,Sushi Restaurant,Mediterranean Restaurant,Japanese Restaurant,Yoga Studio,Ramen Restaurant,Park,Mexican Restaurant,Martial Arts School,Liquor Store,Italian Restaurant
3,Christie,Italian Restaurant,Restaurant,Bakery,Café,Indian Restaurant,Mexican Restaurant,Coffee Shop,Sushi Restaurant,South American Restaurant,Caribbean Restaurant
4,Church and Wellesley,Coffee Shop,Japanese Restaurant,Gay Bar,Sushi Restaurant,Men's Store,Restaurant,Hotel,Café,Dance Studio,Diner


## 4. Cluster Neighborhoods¶
Run k-means to cluster the neighborhood into 5 clusters.

In [79]:
# set number of clusters
kclusters = 5

Downtown_grouped_clustering = Downtown_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(Downtown_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([2, 2, 1, 1, 1, 2, 2, 4, 2, 1], dtype=int32)

#### Let's create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.

In [80]:
# add clustering labels
Downtown_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

Downtown_merged = DownTown_data

# merge manhattan_grouped with manhattan_data to add latitude/longitude for each neighborhood
Downtown_merged = Downtown_merged.join(Downtown_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

Downtown_merged.head() # check the last columns!

,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.659743,-79.361561,0,Coffee Shop,Restaurant,Thai Restaurant,Animal Shelter,Indian Restaurant,Pub,Beer Store,Performing Arts Venue,Food Truck,Pool
1,M5B,Downtown Toronto,"Garden District, Ryerson",43.657940,-79.375620,4,Coffee Shop,Hotel,Sandwich Place,Café,Japanese Restaurant,Restaurant,Hookah Bar,Ramen Restaurant,Pizza Place,Middle Eastern Restaurant
2,M5C,Downtown Toronto,St. James Town,43.671100,-79.373590,4,Coffee Shop,Grocery Store,Pie Shop,Japanese Restaurant,Filipino Restaurant,Caribbean Restaurant,Food & Drink Shop,Bus Stop,Market,Breakfast Spot
3,M5E,Downtown Toronto,Berczy Park,43.648110,-79.375170,2,Coffee Shop,Italian Restaurant,Restaurant,Bakery,Café,Japanese Restaurant,Seafood Restaurant,Gastropub,Gym,Hotel
4,M5G,Downtown Toronto,Central Bay Street,43.665283,-79.387556,1,Sushi Restaurant,Mediterranean Restaurant,Japanese Restaurant,Yoga Studio,Ramen Restaurant,Park,Mexican Restaurant,Martial Arts School,Liquor Store,Italian Restaurant


### Finally, let's visualize the resulting clusters

In [82]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(Downtown_merged['Latitude'], Downtown_merged['Longitude'], Downtown_merged['Neighborhood'], Downtown_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters